In [1]:
# libraries importation


import pandas as pd
import numpy as np
import os
import sys
import csv
import arff

In [2]:
# Specify the file path
australian_path = "./datasets/private/1. UCI Repository/Australian/australian.dat"
german_path = "./datasets/private/1. UCI Repository/German/german.data"



In [23]:
# Read a portion of the file to analyze the separator
num_lines_to_read = 10

# Detect the delimiter
with open(australian_path, 'r') as file:
    sample = file.read(num_lines_to_read)
    dialect = csv.Sniffer().sniff(sample)
    delimiter = dialect.delimiter
    
# Print the detected delimiter
print(f"Detected delimiter ({delimiter})")

Detected delimiter ( )


In [3]:
# Specify the delimiter used in the .dat file
delimiter = " "  # Example: tab-separated values

# Specify the list of header names
australian_names = ["A"+str(i) for i in range(1,16)]
german_names = ["Attribute_"+str(i) for i in range(1,21)]


In [25]:
# Read the .dat file using pandas and set the header names
df_australian = pd.read_csv(australian_path, delimiter=delimiter, names=australian_names)


In [4]:
df_german = pd.read_csv(german_path, delimiter=delimiter, names=german_names)


In [26]:
# Do further operations with the DataFrame
df_australian.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,1,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,0,22.67,7.00,2,8,4,0.165,0,0,0,0,2,160,1,0
2,0,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,0,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,1,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1


In [5]:
df_german.head()

,Attribute_1,Attribute_2,Attribute_3,Attribute_4,Attribute_5,Attribute_6,Attribute_7,Attribute_8,Attribute_9,Attribute_10,Attribute_11,Attribute_12,Attribute_13,Attribute_14,Attribute_15,Attribute_16,Attribute_17,Attribute_18,Attribute_19,Attribute_20
A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


In [12]:
[len(df_german[head].unique().tolist()) for head in df_german.columns.to_list()]

[33, 5, 10, 921, 5, 5, 4, 4, 3, 4, 4, 53, 3, 3, 4, 4, 2, 2, 2, 2]

In [14]:
# export data to csv
#df_australian.to_csv(australian_path[:-4]+".csv", index=False)
df_german.to_csv(german_path[:-4]+"csv", index=False)

In [32]:
# Japan

japan_path = "./1. UCI Repository/Japan/crx.data"

# Specify the delimiter used in the .dat file
delimiter = ","  # Example: tab-separated values

# Specify the list of header names
japan_names = ["A"+str(i) for i in range(1,17)]

df_japan = pd.read_csv(japan_path, delimiter=delimiter, names=japan_names)

# Do further operations with the DataFrame
df_japan.to_csv(japan_path[:-4]+".csv", index=False)
df_japan.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [50]:
# Polish

# Specify the path of the ARFF file
arff_paths = [
    "./datasets/private/1. UCI Repository/Polish/data/1year.arff",
    "./1. UCI Repository/Polish/data/2year.arff",
    "./1. UCI Repository/Polish/data/3year.arff",
    "./1. UCI Repository/Polish/data/4year.arff",
    "./1. UCI Repository/Polish/data/5year.arff"
]

for path in arff_paths:
    # Load the ARFF file
    with open(path, "r") as file:
        data = arff.load(file)


    # Convert the data to a pandas DataFrame
    polish_names = [a[0] for a in data['attributes']]
    data1 = {col: val for col, val in zip(polish_names, data['data'])} 
    df = pd.DataFrame(data1)

    # Do further operations with the DataFrame

    df.to_csv(path+".csv", index=False)



In [ ]:
methods = [method for method in dir(arff) if callable(getattr(arff, method))]

# Print the list of methods with descriptions
for method in methods:
    print(f"Method: {method}")
    print(help(getattr(arff, method)))
    print("=" * 80)

In [ ]:
# Japan

japan_path = "./1. UCI Repository/Japan/crx.data"

# Specify the delimiter used in the .dat file
delimiter = ","  # Example: tab-separated values

# Specify the list of header names
japan_names = ["A"+str(i) for i in range(1,17)]

df_japan = pd.read_csv(japan_path, delimiter=delimiter, names=japan_names)

# Do further operations with the DataFrame
df_japan.to_csv(japan_path[:-4]+".csv", index=False)
df_japan.head()

In [57]:
# Specify the file path
path = "./1. UCI Repository/Australian/australian.dat.csv"

df = pd.read_csv(australian_path, delimiter=' ', header=0)
df.head(10)

,1,22.08,11.46,2,4,4.1,1.585,0,0.1,0.2,1.1,2.1,100,1213,0.3
0,0,22.67,7.000,2,8,4,0.165,0,0,0,0,2,160,1,0
1,0,29.58,1.750,1,4,4,1.250,0,0,0,1,2,280,1,0
2,0,21.67,11.500,1,5,3,0.000,1,1,11,1,2,0,1,1
3,1,20.17,8.170,2,6,4,1.960,1,1,14,0,2,60,159,1
4,0,15.83,0.585,2,8,8,1.500,1,1,2,0,2,100,1,1
5,1,17.42,6.500,2,3,4,0.125,0,0,0,0,2,60,101,0
6,0,58.67,4.460,2,11,8,3.040,1,1,6,0,2,43,561,1
7,1,27.83,1.000,1,2,8,3.000,0,0,0,0,2,176,538,0
8,0,55.75,7.080,2,4,8,6.750,1,1,3,1,2,100,51,0
9,1,33.50,1.750,2,14,8,4.500,1,1,4,1,2,253,858,1


In [ ]:
# Specify the file path
path = "./1. UCI Repository/German/german.data"

df = pd.read_csv(australian_path)
df.head(10)